In [404]:
import pandas as pd
df = pd.read_csv('projet.csv')

In [405]:
import numpy as np

# créer une nouvelle colonne de titre
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)


# remplacer les titres rares par des titres plus courants
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',
           'Don': 'Mr', 'Mme': 'Mrs', 'Jonkheer': 'Mr', 'Lady': 'Mrs',
           'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}
df.replace({'Title': mapping}, inplace=True)

df['Title'].value_counts()


title_ages = dict(df.groupby('Title')['Age'].median())

# créer une colonne des âges moyens
df['age_med'] = df['Title'].apply(lambda x: title_ages[x])

df['Age'].fillna(df['age_med'], inplace=True, )
del df['age_med']

In [406]:
#df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'].fillna(method='backfill', inplace=True)

# créer la colonne Family_Size (Parch + SibSp)
df['Family_Size'] = df['Parch'] + df['SibSp']
df['Sex'] = [0 if x=="male" else 1 for x in df["Embarked"]]

print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    1  35.0      0      0   

             Ticket     Fare Cabin Embarked Title  Family_Size  
0         A/5 21171   7.2500   NaN        S    Mr            1  
1          PC 17599  71.2833   C85        C   Mrs            1  
2  STON/O2. 3101282   7.9250   NaN        S  Miss            0  
3            113803  53.1000  C123  

In [407]:
data = df.drop(['Survived' , 'PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , axis = 1)
#Convertir une variable en plusieurs variables
dataset = pd.get_dummies(data , columns = ['Embarked', 'Title']  , drop_first = False) 
properties = list(dataset.columns.values)
X = dataset[properties]
y = df['Survived']
print(X)

     Pclass  Sex   Age  SibSp  Parch     Fare  Family_Size  Embarked_C  \
0         3    1  22.0      1      0   7.2500            1           0   
1         1    1  38.0      1      0  71.2833            1           1   
2         3    1  26.0      0      0   7.9250            0           0   
3         1    1  35.0      1      0  53.1000            1           0   
4         3    1  35.0      0      0   8.0500            0           0   
..      ...  ...   ...    ...    ...      ...          ...         ...   
886       2    1  27.0      0      0  13.0000            0           0   
887       1    1  19.0      0      0  30.0000            0           0   
888       3    1  21.0      1      2  23.4500            3           0   
889       1    1  26.0      0      0  30.0000            0           1   
890       3    1  32.0      0      0   7.7500            0           0   

     Embarked_Q  Embarked_S  Title_Dr  Title_Master  Title_Miss  Title_Mr  \
0             0           1       

In [408]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 ,random_state = 0)

from sklearn.preprocessing import StandardScaler
# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [409]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(X.shape[1],)),
	keras.layers.Dense(50, input_shape=(X.shape[1],) , activation=tf.nn.relu),
    keras.layers.Dense(1,  activation=tf.nn.sigmoid),
])

print(model.summary())

Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 50)                850       
_________________________________________________________________
dense_124 (Dense)            (None, 1)                 51        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_________________________________________________________________
None


In [410]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [411]:
model.fit(X_train,y_train,epochs=10, batch_size=1 , verbose=1)
test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/10
596/596 [==============================] - 4s 5ms/step - loss: 0.5144 - accuracy: 0.7718
Epoch 2/10
596/596 [==============================] - 3s 5ms/step - loss: 0.4299 - accuracy: 0.8305: 0s - loss: 0.4343 - accuracy: 0.82
Epoch 3/10
596/596 [==============================] - 3s 5ms/step - loss: 0.4108 - accuracy: 0.8389
Epoch 4/10
596/596 [==============================] - 3s 5ms/step - loss: 0.4006 - accuracy: 0.8389: 0s - loss: 0.3957 - accuracy: 0. - ETA: 0s - loss: 0.4026 - 
Epoch 5/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3922 - accuracy: 0.8406: 3s - loss: 0.5345 - accura - ETA: 2s - loss: 0.4168 - accu - ETA: 2s - loss: 0.3789 - accuracy: 0. - ETA: 2s - loss: 0.3665  - ETA: 1s - loss: 0.3910  - ETA: 0s - loss: 0.3984 - accuracy
Epoch 6/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3880 - accuracy: 0.8440
Epoch 7/10
596/596 [==============================] - 3s 5ms/step - loss: 0.3838 - accuracy: 0.8473: 2s -
Epoch 8/

In [401]:
import pandas as pd
y_pred = model.predict(X_test)

In [402]:
import pandas as pd
y_pred = model.predict(X_test)
y_pred[0:5]

array([[0.14851058],
       [0.09699962],
       [0.04702443],
       [0.9905346 ],
       [0.8870611 ]], dtype=float32)

In [403]:
y_test.head(5)

495    0
648    0
278    0
31     1
255    1
Name: Survived, dtype: int64